# 1. Import Modules

In [1]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import datetime

# 2. Read Data

In [2]:
df_M_UDN_file = pd.read_csv('SW_MoneyUDN_KeyWords.csv')

In [3]:
df_M_UDN_file

,URL
0,https://money.udn.com/money/story/5613/7741464
1,https://money.udn.com/money/story/5612/6448041
2,https://money.udn.com/money/story/11074/7741743
3,https://money.udn.com/money/story/123669/7635605
4,https://money.udn.com/money/story/123742/7696251
5,https://money.udn.com/money/story/5607/7755966
6,https://money.udn.com/money/story/5612/7468607
7,https://money.udn.com/money/story/5930/7225331
8,https://money.udn.com/money/story/5613/7741464
9,https://money.udn.com/money/story/5612/7766178...


# 3. Start to Scrape and Export Data

In [34]:
url_number = 0
today_date = datetime.date.today()
content_id = [] # 這邊不會用到
lst_content_introduction = [] # 這邊不會用到
df_money_UDN_sw_all = pd.DataFrame()

for URL in df_M_UDN_file['URL']:
    time.sleep(5)
    url = URL
    print(url)
    headers={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"}
    response = requests.get(url, headers = headers)
    m_udn_html = BeautifulSoup(response.text, "lxml")
        
    # --- Begin 大標題 ---
    
    lst_title_h1 = []
    title_h1 = m_udn_html.find_all("h1")  # 搜尋所有h1
    for title in title_h1:
        print(title.getText())  # 只提取文字部分
        lst_title_h1.append(title.getText())

    # --- End 大標題 ---
    
    # --- Begin 次標題 ---
    
    lst_title_h2 = []
    title_h2 = m_udn_html.find_all("h2", {'class': 'content_title_1'})  # 搜尋所有h2
    for title in title_h2:
        print(title.getText())  # 只提取文字部分
        lst_title_h2.append(title.getText())

    # --- End 次標題 ---
        
    # --- Begin 文章分類 Main and sub ---
    
    lst_content_class_main = []
    lst_content_class_sub = []
    content_class_main = m_udn_html.find_all("li", {"class":"breadcrumb__item"}) 
    # print(content_class_main)
    class_num = 0

    for item in content_class_main:  # r: 一個li區塊
        if class_num == 1:
            lst_content_class_main.append(item.getText().replace('\n',''))
            print(item.getText().replace('\n',''))
        elif class_num == 2:
            lst_content_class_sub.append(item.getText().replace('\n',''))
            print(item.getText().replace('\n',''))

        class_num += 1
        
    # --- End 文章分類 Main and sub ---
    
    content_all = m_udn_html.find("div", {"class":"article-layout-wrapper"})  # 文章內容
    
#     if len(content_all) > 1:
#         content_first = content_all.copy()[0]
#         print("type of content_first", type(content_first))
    
    # --- Begin 文章發佈日 ---
        
    lst_content_date = []

#     for content in content_all:
    if content_all:
        
#         ct_date  = content.find("time", {"class":"article-body__time"})  # 文章發佈日
        
        ct_date  = content_all.find("time", {"class":"article-body__time"})  # 文章發佈日

        print("ct_date:", ct_date)
        lst_content_date.append(ct_date.getText()[:10])
        print(ct_date.getText()[:10])
        
    # --- End 文章分類及文章發佈日 ---
    
    # --- Begin 前言 ---
    
    # Money UDN 多數文章沒有這段，直接歸在內文

    # --- End 前言 ---
    
    # --- Begin 內文 ---

    lst_content_all = []

#     for content in content_all:
        
    if content_all:
#         ct_all = content.find("section", {"class":"article-body__editor"})
        ct_all = content_all.find("section", {"class":"article-body__editor"})
        
        lst_content_all.append(ct_all.getText())
#         print(ct_all.getText()
    
    # --- End 內文 ---
    
    # --- Begin 標籤 ---
    
    lst_content_tag = []
    content_tag = m_udn_html.find_all("li", {"class":"article-keyword__item"})  # 標籤
    tag_num = 0

    for tag_num in range(int(len(content_tag)/2)):
        lst_content_tag.append(content_tag[tag_num].getText().replace('\n',''))
        print(content_tag[tag_num].getText().replace('\n',''))
        print(tag_num)
        tag_num += 1

    # --- End 標籤 ---
    
    
    
    # List 轉 Series
    series_id = pd.Series(content_id)
    series_title_h1 = pd.Series(lst_title_h1)
    
    lst_title_h2 = ",".join(lst_title_h2) # 把list中的資料用","join起來
    series_title_h2 = pd.Series(lst_title_h2)
    
    series_content_class_main = pd.Series(lst_content_class_main)
    series_content_class_sub = pd.Series(lst_content_class_sub)
    series_content_date = pd.Series(lst_content_date)
    series_content_all = pd.Series(lst_content_all)
    
    lst_content_tag = ",".join(lst_content_tag) # 把list中的資料用","join起來
    series_content_tag = pd.Series(lst_content_tag)
    
    money_UDN_column_name = ['content_id'
                            ,'content_title_h1'
                            ,'content_title_h2'
                            ,'content_class_main'
                            ,'content_class_sub'
                            ,'content_date_time'
                            ,'content_all'
                            ,'content_tag']
    
    # Series 放到 DataFrame
    df_money_UDN_sw_keyword = pd.concat([series_id
                                        ,series_title_h1
                                        ,series_title_h2
                                        ,series_content_class_main
                                        ,series_content_class_sub
                                        ,series_content_date
                                        ,series_content_all
                                        ,series_content_tag]
                                        ,axis = 1)
    df_money_UDN_sw_keyword.columns = money_UDN_column_name
    
    url_number += 1
    
    df_money_UDN_sw_all = pd.concat([df_money_UDN_sw_all, df_money_UDN_sw_keyword], ignore_index=True)
    
print("----------------- Finish ------------------")

df_money_UDN_sw_all.to_csv(f'{today_date}_MoneyUDN_SimilarWeb_Keyword.csv', index = False, encoding = 'UTF-8-sig')

https://money.udn.com/money/story/5613/7741464
整理包／2024換新鈔時間、地點一次看！Google 換鈔地圖查最近的銀行、ATM 也可領
換鈔時間：2月1日（周四）至2月7日（周三）
換鈔地點
Google Map新鈔地圖一鍵導航
換鈔限制
不想排隊換新鈔？ATM也可以領
數位紅包怎麼用？
金融
金融脈動
ct_date: <time class="article-body__time">2024/02/06 18:50:47</time>
2024/02/06
華南銀行
0
ATM
1
手機
2
央行
3
統一超商
4


C:\Users\JoeJian\AppData\Local\Temp\ipykernel_46996\2342412729.py:116: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series_id = pd.Series(content_id)


https://money.udn.com/money/story/5612/6448041
一顆電池，讓賓士、軟銀都看中！輝能CEO：我從躲在棉被痛哭到跑贏全球
產業
產業熱點
ct_date: <time class="article-body__time">2022/07/09 03:21:56</time>
2022/07/09
電動車
0
戴姆勒
1
美元
2


C:\Users\JoeJian\AppData\Local\Temp\ipykernel_46996\2342412729.py:116: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series_id = pd.Series(content_id)


https://money.udn.com/money/story/11074/7741743
安國加速轉型 衝先進製程
證券
櫃買動態
ct_date: <time class="article-body__time">2024/01/30 00:46:17</time>
2024/01/30
半導體
0
矽智財
1
晶片
2


...

C:\Users\JoeJian\AppData\Local\Temp\ipykernel_46996\2342412729.py:116: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series_id = pd.Series(content_id)


https://money.udn.com/money/story/5613/7741464
整理包／2024換新鈔時間、地點一次看！Google 換鈔地圖查最近的銀行、ATM 也可領
換鈔時間：2月1日（周四）至2月7日（周三）
換鈔地點
Google Map新鈔地圖一鍵導航
換鈔限制
不想排隊換新鈔？ATM也可以領
數位紅包怎麼用？
金融
金融脈動
ct_date: <time class="article-body__time">2024/02/06 18:50:47</time>
2024/02/06
華南銀行
0
ATM
1
手機
2
央行
3
統一超商
4


C:\Users\JoeJian\AppData\Local\Temp\ipykernel_46996\2342412729.py:116: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series_id = pd.Series(content_id)


https://money.udn.com/money/story/5612/7766178?from=edn_subcatelist_cate
七都之外房價...屏東五年狂漲36%居冠 這2縣市卻回跌了
房市
房市話題
ct_date: <time class="article-body__time">2024/02/12 10:46:08</time>
2024/02/12
屏東
0
房價
1
苗栗
2
----------------- Finish ------------------


C:\Users\JoeJian\AppData\Local\Temp\ipykernel_46996\2342412729.py:116: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series_id = pd.Series(content_id)


In [35]:
df_money_UDN_sw_all

,content_id,content_title_h1,content_title_h2,content_class_main,content_class_sub,content_date_time,content_all,content_tag
0,NaN,整理包／2024換新鈔時間、地點一次看！Google 換鈔地圖查最近的銀行、ATM 也可領,"換鈔時間：2月1日（周四）至2月7日（周三）,換鈔地點,Google Map新鈔地圖一鍵導航...",金融,金融脈動,2024/02/06,\n\n\r\n金龍年將至，換新鈔過好年！經濟日報整理2024年換新鈔時間、地點、方法等資訊...,"華南銀行,ATM,手機,央行,統一超商"
1,NaN,一顆電池，讓賓士、軟銀都看中！輝能CEO：我從躲在棉被痛哭到跑贏全球,,產業,產業熱點,2022/07/09,\n\n\n\n編按：固態電池製造商輝能2023年5月11日宣布，2030年前將投資52億歐...,"電動車,戴姆勒,美元"
2,NaN,安國加速轉型 衝先進製程,,證券,櫃買動態,2024/01/30,\n\nIC設計業者安國（8054）近年積極轉型，藉由入股星河半導體，順勢取得在台積電3奈米...,"半導體,矽智財,晶片"
3,NaN,誰說日本沒有早餐店？東京必訪3間日式早餐 別錯過米其林推薦柴魚飯,,品味,味覺饗宴,2023/12/12,\n\n\r\n去東京旅遊時，如果住宿的飯店沒有附早餐，剛好可以享受3家東京的獨特早餐店，除...,
4,NaN,實支實付亂象 1／金管會推醫療險改革 意外「引爆」實支實付之亂 亂象根源出在哪？,"實支實付之亂 保險公司趁機賺錢？ ,國外少有亂象 實支實付直接理賠醫院,損害填補定義不清 亂...",要聞,深度報導,2024/01/09,\n\n\r\n金管會說要改革實支實付醫療險，落實損害填補原則，但怎麼改都還沒講，市場就掀起...,"醫療險,金管會,防疫保單,發燒話題"
5,NaN,緯創看好 AI 營收跳增三位數 股價改寫六個月新高,,證券,市場焦點,2024/02/05,\n\n代工大廠緯創（3231）看好今年緯創AI營收會跳增「三位數」百分比，激勵今（5）日金...,"緯創,AI,三大法人"
6,NaN,台灣奈微光驚傳技轉大陸？國發基金回應了,,產業,產業熱點,2023/09/27,\n\n\r\n研發出全球第一顆「中遠紅外光4合1矽光子晶片」的台灣奈微光公司，4年前申請國...,"國發基金,投資,晶片"
7,NaN,亞昕插旗台中科博館特區推案 鎖定剛性自住及換屋客層,,房市,房市話題,2023/06/09,\n\n《平均地權條例》修正案即將上路，自住及換屋族群重回主場，備受高端置產客青睞的台中市北...,"建築,植物,台中市"
8,NaN,整理包／2024換新鈔時間、地點一次看！Google 換鈔地圖查最近的銀行、ATM 也可領,"換鈔時間：2月1日（周四）至2月7日（周三）,換鈔地點,Google Map新鈔地圖一鍵導航...",金融,金融脈動,2024/02/06,\n\n\r\n金龍年將至，換新鈔過好年！經濟日報整理2024年換新鈔時間、地點、方法等資訊...,"華南銀行,ATM,手機,央行,統一超商"
9,NaN,七都之外房價...屏東五年狂漲36%居冠 這2縣市卻回跌了,,房市,房市話題,2024/02/12,\n\n\r\n近年來全台房價高漲，不僅主要都會區，七都以外的縣市中古屋房價也多有明顯漲幅，...,"屏東,房價,苗栗"
